# Library

In [2]:
# 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import sys
# check whether run in Colab
if 'google.colab' in sys.modules:
    print('Running in Colab.')
    !pip3 install timm==0.5.4 

Running in Colab.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from __future__ import print_function
from __future__ import division

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torchvision.transforms as T

import os
import pandas as pd
import numpy as np

import json
import csv
import cv2 # for image load

from PIL import Image

import albumentations as A
import albumentations.pytorch
import torchvision.transforms as transforms
from torch.utils.data import random_split

import timm

import random
import plotly.express as px # for grap
from timm.data import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD

from sklearn.metrics import f1_score # for f1 score

from tqdm import tqdm # for progress bar

In [5]:
avail_pretrained_models = timm.list_models(pretrained=True)
len(avail_pretrained_models), avail_pretrained_models

(592,
 ['adv_inception_v3',
  'bat_resnext26ts',
  'beit_base_patch16_224',
  'beit_base_patch16_224_in22k',
  'beit_base_patch16_384',
  'beit_large_patch16_224',
  'beit_large_patch16_224_in22k',
  'beit_large_patch16_384',
  'beit_large_patch16_512',
  'botnet26t_256',
  'cait_m36_384',
  'cait_m48_448',
  'cait_s24_224',
  'cait_s24_384',
  'cait_s36_384',
  'cait_xs24_384',
  'cait_xxs24_224',
  'cait_xxs24_384',
  'cait_xxs36_224',
  'cait_xxs36_384',
  'coat_lite_mini',
  'coat_lite_small',
  'coat_lite_tiny',
  'coat_mini',
  'coat_tiny',
  'convit_base',
  'convit_small',
  'convit_tiny',
  'convmixer_768_32',
  'convmixer_1024_20_ks9_p14',
  'convmixer_1536_20',
  'convnext_base',
  'convnext_base_384_in22ft1k',
  'convnext_base_in22ft1k',
  'convnext_base_in22k',
  'convnext_large',
  'convnext_large_384_in22ft1k',
  'convnext_large_in22ft1k',
  'convnext_large_in22k',
  'convnext_small',
  'convnext_tiny',
  'convnext_xlarge_384_in22ft1k',
  'convnext_xlarge_in22ft1k',
  'c

# Get Dataset

In [6]:
# MY_DIR: 자기 이미지 압축파일 있는 경로 넣기
MY_DIR = '/content/drive/MyDrive/ML/DATA302/dataset'

FAKE_IMG_PATH = '/content/TestSet/'
REAL_IMG_PATH = '/content/REAL/'

In [7]:
os.system(f"unzip {MY_DIR}/TestSet.zip -d /content/")
!mkdir "tars"
os.system(f"tar -xvf {MY_DIR}/ILSVRC2012_img_train_t3.tar -C /content/tars")

!mkdir "REAL"
dir_list = os.listdir("/content/tars")
for dir in dir_list:
  os.system(f"tar -xvf /content/tars/{dir} -C /content/REAL")

mkdir: cannot create directory ‘tars’: File exists
mkdir: cannot create directory ‘REAL’: File exists


In [8]:
# REAL 디렉토리 안에 아무것도 안뜨면 이 코드 한번 돌려보셈
os.listdir("/content/REAL")

['n02096585_664.JPEG',
 'n02113712_1147.JPEG',
 'n02087046_81.JPEG',
 'n02110627_6811.JPEG',
 'n02089973_3119.JPEG',
 'n02107312_2967.JPEG',
 'n02085620_7613.JPEG',
 'n02086910_7321.JPEG',
 'n02111500_9264.JPEG',
 'n02091244_100.JPEG',
 'n02105641_5980.JPEG',
 'n02093859_1936.JPEG',
 'n02102040_3207.JPEG',
 'n02100236_3025.JPEG',
 'n02096585_865.JPEG',
 'n02097658_2182.JPEG',
 'n02086646_1930.JPEG',
 'n02102973_652.JPEG',
 'n02097209_597.JPEG',
 'n02097130_3489.JPEG',
 'n02101006_6152.JPEG',
 'n02108551_13042.JPEG',
 'n02091635_3990.JPEG',
 'n02096437_2310.JPEG',
 'n02090721_4326.JPEG',
 'n02115641_12836.JPEG',
 'n02090622_6166.JPEG',
 'n02115641_4851.JPEG',
 'n02106030_15169.JPEG',
 'n02106166_4450.JPEG',
 'n02089078_222.JPEG',
 'n02109047_31376.JPEG',
 'n02113023_6177.JPEG',
 'n02113023_3687.JPEG',
 'n02097209_3936.JPEG',
 'n02096051_3796.JPEG',
 'n02107908_1235.JPEG',
 'n02101388_6797.JPEG',
 'n02108089_149.JPEG',
 'n02099849_3451.JPEG',
 'n02104365_7409.JPEG',
 'n02090379_364.JPEG'

# Env

In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

random.seed(42)

model_list = {
    "convnext_base_in22ft1k" : {
        "model" : "convnext_base_in22ft1k", 
        "input_size" : (224, 224),
        "classifier_in_feature" : 1024
        }
}

optimizer_list = ["adam", "sgd"]

model_name = "convnext_base_in22ft1k"

CONFIG = {
    "batch_size" : 16,
    "epoch" : 200,
    "learning_rate" : 1e-4,
    "input_size" : model_list[model_name]["input_size"],
    "backbone" : model_list[model_name]["model"],
    "classifier_in_feature": model_list[model_name]["classifier_in_feature"],
    "device" : device,
    "patience" : 10,
    "optimizer" : optimizer_list[0],
    "input_norm_mean" : IMAGENET_DEFAULT_MEAN,
    "input_norm_std" : IMAGENET_DEFAULT_STD
}

batch_size = CONFIG["batch_size"]
epochs = CONFIG["epoch"]
learning_rate = CONFIG["learning_rate"]
input_size = CONFIG["input_size"]
backbone = CONFIG["backbone"]
classifier_in_feature = CONFIG["classifier_in_feature"]
device = CONFIG["device"]
patience = CONFIG["patience"]
optimizer = CONFIG["optimizer"]
input_norm_mean = CONFIG["input_norm_mean"]
input_norm_std = CONFIG["input_norm_std"]

Using cpu device


# Read the Data

In [10]:
class CustomDataset(Dataset):
    def __init__(self, FAKE_path_label, REAL_path_label, input_size, transform=None, isTest=False):
        if isTest:
            self.FAKE_path_label = FAKE_path_label
            self.REAL_path_label = REAL_path_label
            self.input_size = input_size
            self.transform = transform
            self.isTest = isTest
            self.img_list = []

            for i in range(len(FAKE_path_label)):
                img_path = FAKE_path_label[i][0]
                image = Image.open(img_path)
                if self.transform:
                    image = self.transform(image=np.array(image))['image']
                self.img_list.append((image, 0))

            for i in range(len(REAL_path_label)):
                img_path = REAL_path_label[i][0]
                image = Image.open(img_path)
                if self.transform:
                    image = self.transform(image=np.array(image))['image']
                self.img_list.append((image, 1))
        else:
          # TODO
            pass

    def __len__(self):
        return len(self.img_list)

    def __getitem__(self, idx):
        image, label = self.img_list[idx]

        return image, label #label type: int

In [24]:
FAKE_path_label = []
REAL_path_label = []

# Get Fake Data
with open(os.path.join(FAKE_IMG_PATH, "operations.csv"), "r") as operations_csv:
    row = csv.reader(operations_csv, delimiter=',')
    for i, col in enumerate(row):
        if col[6] == "FALSE" and int(col[1]) >= 224: # col[6]: label, col[1]: cropsize
            FAKE_path_label.append((FAKE_IMG_PATH + col[0], 0)) # col[0]: src, 0 for false
# FAKE_path_label = FAKE_path_label[1:]
print(FAKE_path_label[0])
print(len(FAKE_path_label))

# Get Real Data
REAL_path_label = [(os.path.join(REAL_IMG_PATH, path), 1) for path in os.listdir(REAL_IMG_PATH)]
print(REAL_path_label[0])
print(len(REAL_path_label))

A_transform = A.Compose([
    A.PadIfNeeded(*input_size),
    albumentations.augmentations.crops.transforms.CenterCrop(*input_size),
    albumentations.augmentations.transforms.Normalize(
        mean=input_norm_mean, 
        std=input_norm_std
        ),
    albumentations.pytorch.transforms.ToTensorV2()
])

FAKE_path_label = random.sample(FAKE_path_label, k=1000)
REAL_path_label = random.sample(REAL_path_label, k=1000)

test_dataset = CustomDataset(FAKE_path_label, REAL_path_label, input_size=input_size, transform=A_transform, isTest=True)
test_dataloader = DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    shuffle=False
)

('/content/TestSet/biggan_256/biggan_002_230631.png', 0)
7960
('/content/REAL/n02096585_664.JPEG', 1)
20580


# Model

In [15]:
class Model(nn.Module):
    def __init__(self, backbone, classifier_in_feature, pretrained=True):
        super(Model, self).__init__()
        self.backbone = timm.create_model(backbone, pretrained=pretrained)
        self.backbone.reset_classifier(0)
        self.classifier = nn.Linear(
                in_features=classifier_in_feature,
                out_features=2
            )
        
        for param in self.backbone.parameters():
            param.requires_grad=False

    def forward(self, input):
        output = self.backbone(input) # bs 1 1000
        output = self.classifier(output)
        return output


model = Model(backbone, classifier_in_feature, pretrained=True)
model = model.to(device)

loss_fn = nn.CrossEntropyLoss()

if optimizer == "adam":
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 
elif optimizer == "sgd":
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
print(model)

Model(
  (backbone): ConvNeXt(
    (stem): Sequential(
      (0): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
      (1): LayerNorm2d((128,), eps=1e-06, elementwise_affine=True)
    )
    (stages): Sequential(
      (0): ConvNeXtStage(
        (downsample): Identity()
        (blocks): Sequential(
          (0): ConvNeXtBlock(
            (conv_dw): Conv2d(128, 128, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=128)
            (norm): LayerNorm((128,), eps=1e-06, elementwise_affine=True)
            (mlp): Mlp(
              (fc1): Linear(in_features=128, out_features=512, bias=True)
              (act): GELU(approximate='none')
              (drop1): Dropout(p=0.0, inplace=False)
              (fc2): Linear(in_features=512, out_features=128, bias=True)
              (drop2): Dropout(p=0.0, inplace=False)
            )
            (drop_path): Identity()
          )
          (1): ConvNeXtBlock(
            (conv_dw): Conv2d(128, 128, kernel_size=(7, 7), stride=(1, 1)

# Test

In [26]:
@torch.no_grad()
def test_model(device, model, test_loader):
    pred_ans = []
    gt_ans = []
    corrects = 0

    model.eval()
    model = model.to(device)

    for x, gt in tqdm(test_loader):  # tqdm: progress bar 표시
        x, gt = x.to(device), gt.to(device)
        outputs = model(x)
        _, preds = torch.max(outputs, 1)
        
        pred_ans += preds.tolist()
        gt_ans += gt.tolist()
        corrects += torch.sum(preds == gt.data)

    # accuracy 출력
    print(f"total accuracy: {corrects / len(test_loader.dataset)}")

    return pred_ans, gt_ans


In [17]:
# confusion matrix 시각화
import matplotlib.pyplot as plt
import itertools    # confusion matrix에서 사용
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(label, pred, target_names=None, labels=True):
    cm = confusion_matrix(label, pred)
    accuracy = np.trace(cm) / float(np.sum(cm))

    cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(9, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.colorbar()
    thresh = cm.max() / 2

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names)
        plt.yticks(tick_marks, target_names)

    if labels:
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            plt.text(j, i, "{:,}".format(cm[i, j]), horizontalalignment="center",
                     color="white" if cm[i,j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

In [ ]:
pred_ans, gt_ans = test_model(device, model, test_dataloader)
plot_confusion_matrix(gt_ans, pred_ans)

 59%|█████▉    | 74/125 [12:37<08:39, 10.19s/it]

In [ ]:
correct = [0,0,0,0] # 1,1 0,0 1,0 0,1

pred_ans = test_model(device, model, test_dataloader)

for (x, y) in pred_ans:
    if x == 1 and y == 1:
        correct[0] += 1
    elif x == 0 and y == 0:
        correct[1] += 1
    elif x == 1 and y == 0:
        correct[2] += 1
    elif x == 0 and y == 1:
        correct[3] += 1
    else:
        print(f"x:{x}, x type:{type(x)}, y:{y}, y type:{type(y)}")


print(pred_ans)
print(correct)

  0%|          | 0/125 [00:00<?, ?it/s]


TypeError: ignored